### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [63]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [64]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [65]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [66]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [67]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [68]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [69]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [70]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [71]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [72]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [73]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [74]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [75]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [76]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [77]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [78]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [79]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [80]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [81]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [82]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


### Resolución del Desafío

**1**. Vectorizar documentos.

In [83]:
import random

rnd_idxs = random.sample(range(0, len(newsgroups_train.data)), 5)

for idx in rnd_idxs:
    # similaridad coseno con todos los documentos de train
    cos_sim = cosine_similarity(X_train[idx], X_train)[0]
    # los 5 documentos más similares:
    most_sim_indices = np.argsort(cos_sim)[::-1][1:6]
    # demo del texto elegido:
    selected_text_demo = newsgroups_train.data[idx][:150].replace("\n", " ").strip()
    print(f"Texto: '{selected_text_demo}'")
    # el documento original pertenece a la clase:
    print(f"Clase del documento seleccionado al azar: {newsgroups_train.target_names[y_train[idx]]}")
    # y los 5 más similares son de las clases:
    print("Documentos con mayor cosine_similarity: ")
    for i in most_sim_indices:
        print(f"Indice: '{i}'")
        text_demo = newsgroups_train.data[i][:150].replace("\n", " ").strip()
        print(f"Texto: '{text_demo}'")
        print(f"Clase: {newsgroups_train.target_names[y_train[i]]} - cosine_similarity: {cos_sim[i]}")
    print("--------------")

Texto: '[deleted]  I'm sorry if I'm misunderstanding your post, but DRAM *does not* have to be refreshed on *each access cycle*.  So cycle time does *not* hav'
Clase del documento seleccionado al azar: comp.sys.ibm.pc.hardware
Documentos con mayor cosine_similarity: 
Indice: '1416'
Texto: 'This is an official RFD for the creation of a new newsgroup for the general discussion of the Microsoft Access RDMS.  NAME: COMP.DATABASES.ACCESS  MOD'
Clase: comp.os.ms-windows.misc - cosine_similarity: 0.23538719866547453
Indice: '10569'
Texto: 'This is the 2nd Reguest for Discussion ( RFD ) for the creation of  comp.databases.ms-access   The discussion period started on April 6 and it will la'
Clase: comp.os.ms-windows.misc - cosine_similarity: 0.22706083032657617
Indice: '8493'
Texto: 'Rick Anderson replied to my letter with...  ra> In article <C5ELp2.L0C@acsu.buffalo.edu>, ra> ra> >     Well, Jason, it's heretical in a few ways. The'
Clase: talk.religion.misc - cosine_similarity: 0.2212674550073

La sección de código precedente toma 5 entradas del dataset de train al azar y calcula los 5 documentos más similares empleando la función cosine_similarity. Se observa que en general los temas son bastante similares. Existen oportunidades en las que algún tema puede parecer algo desconexo, pero en general se observa una buena tasa de similaridad en las temáticas arrojadas por la función cosine_similarity.

**2**. Mejora de modelos de clasificación Naïve Bayes.

Se ejecutará un GridSearchCV a fin de encontrar el valor óptimo para el hiperparámetro "alpha" de MultinomialNB().

* alpha: Este hiperparámetro controla el suavizado de Laplace. Éste se utiliza para prevenir la probabilidad de que una característica dada no aparezca en una clase dada durante el entrenamiento, lo que puede conducir a estimaciones de probabilidad cero y problemas asociados durante la clasificación.

In [84]:
from sklearn.model_selection import GridSearchCV

# Definir el modelo
mnb_model = MultinomialNB()

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {'alpha': [0.01, 0.05, 0.1, 0.5, 1.0]}

# Realizar búsqueda grid
grid_search = GridSearchCV(mnb_model, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_alpha = grid_search.best_params_['alpha']

# Entrenar modelo final con los mejores hiperparámetros
final_mnb_model = MultinomialNB(alpha=best_alpha)
final_mnb_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba y calcular F1-score
y_pred = final_mnb_model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')

print("Mejor valor de alpha:", best_alpha)
print("F1-score en conjunto de entrenamiento:", f1)

Mejor valor de alpha: 0.01
F1-score en conjunto de entrenamiento: 0.682861129525057


Se observa que el F1-score para el caso del empleo de GridSearchCV ha aumentado notoriamente, comparado con el modelo MultinomialNB() de ejemplo, en donde se deja el parámetro "alpha" por default. (F1-score mejora de 0.58 a 0.68 aproximadamente)

Se procede ahora a probar con el modelo ComplementNB(). Se plantea también la búsqueda de hiperparámetros óptimos a través de GridSearchCV.
Para el caso de ComplementNB aparecen dos nuevos hiperparámetros.

* "fit_prior" indica si se deben aprender las probabilidades a priori de las clases o si se deben utilizar distribuciones uniformes.

* "norm" controla si se debe aplicar la normalización de características de la frecuencia dentro del algoritmo.

In [94]:
# Definir el modelo
cnb_model = ComplementNB()

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {'alpha': [0.01, 0.1, 0.5, 1.0],
              'fit_prior': [True, False],
              'norm': [True, False]}

# Realizar búsqueda grid
grid_search = GridSearchCV(cnb_model, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_

# Entrenar modelo final con los mejores hiperparámetros
final_cnb_model = ComplementNB(**best_params)
final_cnb_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba y calcular F1-score
y_pred = final_cnb_model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')

print("Mejores hiperparámetros:", best_params)
print("F1-score en conjunto de entrenamiento:", f1)

Mejores hiperparámetros: {'alpha': 0.1, 'fit_prior': True, 'norm': False}
F1-score en conjunto de entrenamiento: 0.6953652590540836


Se observa una mejora de ComplementNB() respecto de MultinomialNB(). De todas formas, la diferencia no es tan grande (0.695 de ComplementNB versus 0.682 de MultinomialNB).

**3**. Estudio de similaridad entre palabras.

Transponemos la matriz de train documento-término para obtener una matriz término-documento. Al ver el shape de X_train, podemos ver que existen 11.314 documentos y 101.631 términos.

In [86]:
X_train.shape

(11314, 101631)

In [87]:
X_train_term_doc = X_train.T
X_train_term_doc.shape

(101631, 11314)

Guardo los términos en un array aparte para acceder luego y mostrar su valor.

In [88]:
feature_term_names = tfidfvect.get_feature_names_out()
# feature_term_names = idx2word

Al existir varios términos numéricos, abreviaciones y cadenas de texto que no son palabras del idioma inglés, se decide seleccionar previamente 5 palabras con sentido y significado en inglés. Se tomó esta decisión ya que en el enunciado del ejercicio no especifica que la selección de los términos deba ser al azar.

In [89]:
# rnd_idxs = random.sample(range(0, X_train_term_doc.shape[0]), 5)
# Comentado, ya que hay términos numéricos, abreviaciones, etc, y no hace sentido tomar valores al azar

# Se analizarán 5 términos puntuales
# 29412 conspiracy
# 36688 electrically
# 57469 loss
# 61750 milestone
# 80816 scheduled
rnd_idxs = [29412, 36688, 57469, 61750, 80816]

for idx in rnd_idxs:
    # similaridad coseno con todos los términos de train
    cos_sim_term = cosine_similarity(X_train_term_doc[idx], X_train_term_doc)[0]
    # los 5 términos más similares:
    most_sim_term_indices = np.argsort(cos_sim_term)[::-1][1:6]
    # el término original es:
    print(f"Término seleccionado: {feature_term_names[idx]}")
    # y los 5 términos más similares son:
    print("Términos con mayor cosine_similarity: ")
    for i in most_sim_term_indices:
        print(f"Término: {feature_term_names[i]} - cosine_similarity: {cos_sim_term[i]}")
    print("--------------")

Término seleccionado: conspiracy
Términos con mayor cosine_similarity: 
Término: forthright - cosine_similarity: 0.35395514711982123
Término: suppressing - cosine_similarity: 0.34773112552765983
Término: speculations - cosine_similarity: 0.3343549838207819
Término: torching - cosine_similarity: 0.29099765766506464
Término: quisling - cosine_similarity: 0.29099765766506464
--------------
Término seleccionado: electrically
Términos con mayor cosine_similarity: 
Término: loudest - cosine_similarity: 0.7223425655620517
Término: cretainly - cosine_similarity: 0.5878091012500084
Término: farrady - cosine_similarity: 0.5878091012500084
Término: chancy - cosine_similarity: 0.5818984517207866
Término: piped - cosine_similarity: 0.5372962573982003
--------------
Término seleccionado: loss
Términos con mayor cosine_similarity: 
Término: timer - cosine_similarity: 0.22822950887189108
Término: fredericks - cosine_similarity: 0.20942972763680612
Término: reestablish - cosine_similarity: 0.2059090161

Se observa que para el caso de términos, la similaridad entre el significado de las palabras con mayor cosine_similarity es un tanto más difusa. En algunos casos la similaridad es más clara y en otros casos no tanto.

La similaridad parece darse más por el contexto temático en donde el conjunto de palabras más símiles aparecen, que por el significado puntual en inglés (desde el punto de vista de encontrar sinónimos).